In [1]:
import pickle
import random
import numpy as np
import networkx as nx
from sklearn.decomposition import PCA

import torch

from smartsampling import *
from evaluation import *

In [2]:
seed = 1
device = "cuda" if torch.cuda.is_available() else "cpu"

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if device=="cuda":
    torch.cuda.manual_seed(seed)

In [3]:
node_features = pickle.load(open('data/cora/node_features.pkl', 'rb'))
text_features = node_features.toarray()

pca = PCA(n_components=100)
pca.fit(text_features)
emb_features = pca.transform(text_features)

nnodes = text_features.shape[0]
device = "cuda" if torch.cuda.is_available() else "cpu"

## 3 nodes each walk, strongly favoring 1-hop neighbor

In [4]:
p_space = [9e50] #[0.2, 0.5, 1, 2, 5]
q_space = [9e50] #[0.2, 0.5, 1, 2, 5]
c_space = [3] #[1, 2, 3, 4, 5]

nsamplers = int(nnodes/1)
nwalk = 100
nnega = 5
penalty = 3e-4
dropout = 0.4

lr = 0.005
weight_decay = 0.0005

ratio = 0.1
nfold = 5

In [5]:
lp_train = pickle.load(open('data/cora/lp_train.pkl', 'rb'))
lp_test = pickle.load(open('data/cora/lp_test.pkl', 'rb'))

X_train, y_train, X_test, y_test = pickle.load(open('simulation/train_test.pkl', 'rb'))

# node_adjs = nx.from_scipy_sparse_matrix(pickle.load(open('data/cora/node_adjs.pkl', 'rb')))
# node_labels = pickle.load(open('data/cora/node_labels.pkl', 'rb'))

In [6]:
model = SmartSampling(text_features, emb_features, p_space, q_space, c_space, 
                      nnodes, nsamplers, nnega, lr, weight_decay, dropout, device, 
                      lp_train, X_train, y_train)

In [7]:
embeddings = model.train(0, penalty)
# nc_acc, nc_f1 = nc_evaluate(embeddings, node_labels)
# print('Walk: {}'.format(0), nc_acc, nc_f1)
lp_acc, lp_f1 = lp_evaluate(embeddings, lp_test)
print("Walk: {}".format(0), lp_acc, lp_f1)
    
for i in range(1, nwalk+1):
    embeddings = model.train(1, penalty)
    lp_acc, lp_f1 = lp_evaluate(embeddings, lp_test)
    print("Walk: {}".format(i), lp_acc, lp_f1)

Walk: 0 0.6803364879074658 0.7179962894248609
Epoch: 1, Loss: 0.7025, Acc: 0.3871, F1: 0.5582, Gain: 1.0159 || Nonneighbor_Avg: 0.8538, Neighbor_Avg: 1.4073, Time: 10.6846s
Walk: 1 0.49947423764458465 0.6661991584852734
Epoch: 1, Loss: 0.7014, Acc: 0.3871, F1: 0.5582, Gain: 0.8664 || Nonneighbor_Avg: 0.8648, Neighbor_Avg: 1.3940, Time: 10.5219s
Walk: 2 0.49947423764458465 0.6661991584852734
Epoch: 1, Loss: 0.6972, Acc: 0.3871, F1: 0.5582, Gain: 0.8504 || Nonneighbor_Avg: 0.8623, Neighbor_Avg: 1.3955, Time: 10.3620s
Walk: 3 0.49947423764458465 0.6661991584852734
Epoch: 1, Loss: 0.6951, Acc: 0.3871, F1: 0.5582, Gain: 0.8394 || Nonneighbor_Avg: 0.8405, Neighbor_Avg: 1.3999, Time: 10.5126s
Walk: 4 0.49947423764458465 0.6661991584852734
Epoch: 1, Loss: 0.6910, Acc: 0.3871, F1: 0.5582, Gain: 0.8351 || Nonneighbor_Avg: 0.8545, Neighbor_Avg: 1.4003, Time: 10.4449s
Walk: 5 0.49947423764458465 0.6661991584852734
Epoch: 1, Loss: 0.6899, Acc: 0.3871, F1: 0.5582, Gain: 0.8221 || Nonneighbor_Avg: 0.

Epoch: 1, Loss: 0.6417, Acc: 0.3930, F1: 0.5604, Gain: 0.6260 || Nonneighbor_Avg: 0.8468, Neighbor_Avg: 1.3977, Time: 10.4834s
Walk: 48 0.501577287066246 0.6671348314606742
Epoch: 1, Loss: 0.6431, Acc: 0.3930, F1: 0.5604, Gain: 0.6312 || Nonneighbor_Avg: 0.8250, Neighbor_Avg: 1.4014, Time: 10.4103s
Walk: 49 0.5047318611987381 0.6685432793807178
Epoch: 1, Loss: 0.6437, Acc: 0.3918, F1: 0.5600, Gain: 0.6284 || Nonneighbor_Avg: 0.8335, Neighbor_Avg: 1.4103, Time: 10.2651s
Walk: 50 0.5057833859095688 0.6690140845070423
Epoch: 1, Loss: 0.6432, Acc: 0.3934, F1: 0.5607, Gain: 0.6228 || Nonneighbor_Avg: 0.8194, Neighbor_Avg: 1.4069, Time: 10.9263s
Walk: 51 0.5047318611987381 0.6685432793807178
Epoch: 1, Loss: 0.6434, Acc: 0.3932, F1: 0.5606, Gain: 0.6190 || Nonneighbor_Avg: 0.8408, Neighbor_Avg: 1.4014, Time: 10.9749s
Walk: 52 0.5026288117770767 0.6676036542515812
Epoch: 1, Loss: 0.6411, Acc: 0.3956, F1: 0.5615, Gain: 0.6269 || Nonneighbor_Avg: 0.8604, Neighbor_Avg: 1.3948, Time: 10.9761s
Walk

Epoch: 1, Loss: 0.6385, Acc: 0.3968, F1: 0.5620, Gain: 0.6258 || Nonneighbor_Avg: 0.8497, Neighbor_Avg: 1.3970, Time: 10.5922s
Walk: 96 0.5057833859095688 0.6690140845070423
Epoch: 1, Loss: 0.6409, Acc: 0.3969, F1: 0.5621, Gain: 0.6299 || Nonneighbor_Avg: 0.8711, Neighbor_Avg: 1.3936, Time: 10.4426s
Walk: 97 0.5057833859095688 0.6690140845070423
Epoch: 1, Loss: 0.6394, Acc: 0.3991, F1: 0.5630, Gain: 0.6389 || Nonneighbor_Avg: 0.8390, Neighbor_Avg: 1.3966, Time: 10.3944s
Walk: 98 0.5068349106203995 0.6690190543401552
Epoch: 1, Loss: 0.6448, Acc: 0.3967, F1: 0.5618, Gain: 0.6409 || Nonneighbor_Avg: 0.8379, Neighbor_Avg: 1.4036, Time: 10.3643s
Walk: 99 0.508937960042061 0.6704304869442484
Epoch: 1, Loss: 0.6413, Acc: 0.3968, F1: 0.5619, Gain: 0.6433 || Nonneighbor_Avg: 0.8309, Neighbor_Avg: 1.4036, Time: 10.3430s
Walk: 100 0.5078864353312302 0.6694915254237289
